In [78]:
import pandas as pd
import random
from datasets import Dataset, concatenate_datasets, load_dataset

In [188]:
dataset = pd.read_json("/nas/ucb/shivamsinghal/quality/data/v1.0.1/QuALITY.v1.0.1.htmlstripped.train", lines=True)

In [189]:
def get_lengths(row):
    return len(row["article"])

dataset["article_length"] = dataset.apply(get_lengths, axis=1)
dataset = dataset[dataset["article_length"] < 30000]

In [190]:
dataset = dataset.sample(frac=1)
dataset.shape

(250, 14)

In [191]:
dataset = dataset[0:50]

In [192]:
dataset["questions_list"] = dataset["questions"]

In [193]:
def get_updated_questions_list(row):
    lst = row["questions_list"]
    updated_lst = [l for l in lst if l["writer_label"]==l["gold_label"]]
    question_dict = random.sample(updated_lst, 1)[0]
    label = question_dict["gold_label"]
    options = question_dict["options"]
    correct_option = options[label-1]
    speed_answer = random.sample(question_dict["speed_validation"], 1)[0]["speed_answer"]
    if speed_answer == label:
        options.remove(correct_option)
        incorrect_option = random.sample(options, 1)[0]
        chosen_answer, rejected_answer = correct_option, incorrect_option
        chosen_type, rejected_type = "correct", "incorrect"
    else:
        incorrect_option = options[speed_answer-1]
        chosen_answer, rejected_answer = incorrect_option, correct_option
        chosen_type, rejected_type = "incorrect", "correct"

    return updated_lst, question_dict["question"], correct_option, incorrect_option, chosen_answer, chosen_type, rejected_answer, rejected_type 

In [194]:
dataset[["questions_list", "questions", "correct_statements", "incorrect_statements", "chosen", "chosen_type", "rejected", "rejected_type"]] = dataset.apply(get_updated_questions_list, axis=1, result_type="expand")

In [219]:
dataset = dataset.drop(['article_id', 'set_unique_id', 'batch_num', 'writer_id', 'title', 'year', 'author', 'topic', 'url', 'license', 'article_length', 'questions_list'], axis=1)

In [228]:
dataset.to_csv("quality.csv", index=False)

In [212]:
true = pd.read_csv("TRUE_dataset.csv")

In [226]:
true.columns

Index(['source', 'difficulty', 'choice1', 'choice2', 'choice1_type',
       'choice2_type', 'chosen', 'rejected', 'chosen_type', 'rejected_type',
       'confidence_scores', 'confidence_difficulty', 'decision_time',
       'number_of_clicks', 'scratch_space', 'correct_chosen',
       'raw_number_of_clicks', 'raw_decision_time', 'prompt',
       'prompt_response_group', 'other_info', 'IDs', 'tags', 'tag_IDs',
       'categories', 'correct_statements', 'incorrect_statements'],
      dtype='object')